# 01 – Exploração do Snapshot

Primeiro notebook do **creditlab**.

Nele, vamos:
1. Instanciar o `CreditDataSynthesizer`.
2. Gerar a *safra 0* (snapshot) para alguns grupos homogêneos.
3. Fazer uma exploração inicial: contagem de contratos por grupo, estatísticas descritivas e um histograma de dias em atraso.

⚠️ Este notebook assume que o módulo `credit_data_synthesizer.py` está no mesmo diretório ou em `PYTHONPATH`.

In [ ]:
import sys
from pathlib import Path

# Adiciona o diretório-pai ao sys.path
sys.path.append(str(Path().resolve().parent))

import pandas as pd
import matplotlib.pyplot as plt


from importlib import reload
import credit_data_synthesizer as cds
reload(cds)  # garante versão atual no notebook

from credit_data_synthesizer import (
    CreditDataSynthesizer,
    default_group_profiles,   # função helper já incluída no módulo
)

from credit_data_sampler import (
    TargetSampler
)

## Parâmetros iniciais

In [ ]:
# ----- parâmetros que você quer -----
n_groups = 6
contracts_per_group = 1_000
n_safras = 12
seed = 0

# 1) Obtenha os perfis padrão e pegue só os três primeiros
group_profiles = default_group_profiles(n_groups)

# 2) Instancie o sintetizador
synth = CreditDataSynthesizer(
    group_profiles=group_profiles,
    contracts_per_group=contracts_per_group,
    n_safras=n_safras,
    random_seed=0,
    start_safra = 202001,
    target_ratio=0.08,
)

## Geração da safra 0

In [ ]:
# 3) Gere os dados
df_snapshot, df_panel, df_trace = synth.generate()

print(f"Snapshot gerado com {len(df_snapshot):,} linhas")
print(f"Painel gerado com {len(df_panel):,} linhas")
print(f"Rastro gerado com {len(df_trace):,} linhas")
display(df_snapshot.head())

In [ ]:
df_panel.groupby('grupo_homogeneo')['ever90m12'].mean()*100

In [ ]:
df_panel['nivel_refinanciamento'].describe()

In [ ]:
# balanceando target
sampler = TargetSampler(target_ratio=0.10)
balanced = sampler.fit_transform(df_panel, target_col="ever90m12")

prev = balanced.groupby("safra")["ever90m12"].mean()*100
print("Prev per safra (balanced):")
print(prev.head())

In [ ]:
df_snapshot.groupby('sexo')['ever90m12'].mean()*100

In [ ]:
df_snapshot.groupby('grupo_homogeneo')['ever90m12'].mean()*100

In [ ]:
balanced.groupby('grupo_homogeneo')['ever90m12'].mean()*100

In [ ]:
len(balanced), len(df_panel)

In [ ]:
balanced['safra'].value_counts()

In [ ]:
balanced.groupby(by='safra', as_index=True)['ever90m12'].mean()*100

### Contagem de contratos por `grupo_homogeneo`

In [ ]:
df_snapshot['grupo_homogeneo'].value_counts()

### Estatísticas descritivas das variáveis numéricas

In [ ]:
cols_numericas = balanced.select_dtypes(include='number').columns.drop(['id_contrato', 'id_cliente'])

agg_df = (
    balanced[cols_numericas.tolist() + ['grupo_homogeneo']]
    .groupby('grupo_homogeneo')
    .agg(['min', 'mean', 'max'])
)

# Flatten columns
agg_df.columns = ['_'.join(col) for col in agg_df.columns]
agg_df.reset_index(inplace=True)

# Supondo que agg_df tenha a coluna 'grupo_homogeneo'
agg_df = agg_df.copy()
agg_df['grupo_idx'] = agg_df['grupo_homogeneo'].str.extract(r'GH(\d+)').astype(int)
agg_df = agg_df.sort_values('grupo_idx').drop(columns='grupo_idx')

agg_df.head(10)

### Distribuição de `dias_atraso`

In [ ]:
plt.figure()
balanced['dias_atraso'].hist(bins=30)
plt.xlabel('Dias em atraso')
plt.ylabel('Número de contratos')
plt.title('Histograma de dias em atraso – safra 0')
plt.show()

In [ ]:
df_snapshot.shape